# LangChain을 활용한 뉴스 요약 및 키워드 추출

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

import re
import json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")


`(3) 뉴스 데이터 로드`

In [3]:
# pickle 파일에서 데이터 로드
import pickle

with open("processed_news_articles.pkl", "rb") as f:
    loaded_docs = pickle.load(f)
    print(f"로드된 문서 수: {len(loaded_docs)}")

로드된 문서 수: 69


In [4]:
loaded_docs[0].metadata

{'source': 'https://n.news.naver.com/mnews/article/018/0005968393?sid=105',
 'media_outlet': '이데일리',
 'reporter': '김현아',
 'content': '인공지능(AI) 기업 크라우드웍스가 중국 기업 딥시크와 계약을 체결하고 딥시크의 공식적인 한국어 특화 모델을 개발한다는 보도에 대해 반박했다. 크라우드웍스는 해당 보도 내용이 사실이 아니며, 딥시크와 어떠한 형태로도 직접적인 협업 관계가 없음을 명확히 밝혔다. 크라우드웍스는 일본 법인인 CWJ(크라우드웍스 재팬)를 통해 일본의 협력사 Aicho(아이초)와 협업하고 있으며, Aicho는 최근 딥시크 R1 모델을 기반으로 일본어 특화 모델을 개발한 바 있으며, 크라우드웍스는 이를 바탕으로 한국어 특화 모델을 개발할 예정이다. 이는 딥시크와의 계약이 아니며, Aicho의 일본어 모델을 기반으로 한 한국어 특화 모델 개발에 해당한다고 설명했다. 또한, 크라우드웍스는 딥시크의 개인정보 유출 문제가 B2C 모델에서 발생한 사안이며, 자신들이 Aicho와 공동 개발 중인 한국어 모델은 일반 소비자 대상이 아닌 B2B 모델에 해당한다고 밝혔다. 이 한국어 특화 모델은 주로 해외 시장을 타겟으로 하며, 금융, 공공, 국방 등 보안이 민감한 산업 분야에는 공급될 계획이 없다고 강조했다. 기업의 특별한 니즈가 있을 경우 온프레미스 환경에서의 활용을 검토할 수 있으며, 이는 물리적으로 제한된 네트워크 환경 내에서만 운영돼 외부 네트워크와의 통신 여부를 철저히 검증할 수 있다고 설명했다. 크라우드웍스는 최근 K-콘텐츠와 K-커머스의 글로벌 확산에 따라 한국어에 특화된 AI 모델에 대한 수요가 급증하고 있음을 언급하며, 해외 시장 진출을 목표로 한국어 특화 모델 개발을 추진하고 있다고 밝혔다.'}

---

## **LLM을 활용한 텍스트 분석**

- **LangChain**: LLM 기반 애플리케이션 개발을 위한 프레임워크
- **텍스트 요약 (Summarization)**: 긴 문서를 간결하고 의미 있는 형태로 변환
- **키워드 추출 (Extraction)**: 문서에서 핵심 용어와 개념을 자동으로 식별
- **평가 (Evaluation)**: 생성된 결과의 품질을 정량적/정성적으로 측정

---

### 1. **텍스트 요약 (Summarization)**

- **텍스트 요약의 목적**
    - 긴 문서의 **핵심 내용만 추출**하여 읽기 시간 단축
    - 정보의 **본질적 의미는 유지**하면서 길이 축소
    - 대량의 문서를 빠르게 **스캔하고 분류**하는 데 활용

- **요약의 유형**
    - **추출적 요약 (Extractive)**: 원문에서 중요한 문장을 그대로 선택
    - **생성적 요약 (Abstractive)**: 원문을 이해하고 새로운 문장으로 재작성 ⭐ (LLM 방식)

##### **1) 텍스트 요약 체인**

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chat_models import init_chat_model

# LLM 초기화
llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)

# 요약 프롬프트 템플릿 정의
summarization_prompt = ChatPromptTemplate.from_messages([
    ("system", "다음 텍스트를 간결하게 요약하세요. 핵심 내용만 포함하고 중요한 정보는 유지하세요. (공백 포함 100글자 이내로 요약)"),
    ("human", "{text}")
])

# 요약 체인 생성
summarization_chain = summarization_prompt | llm | StrOutputParser()

# 예시 사용
text = """
LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다. 
이 프레임워크는 LLM을 외부 데이터 소스와 연결하고, 다양한 모듈을 제공하여 
더 강력한 애플리케이션을 만들 수 있게 도와줍니다. 
LangChain은 프롬프트 관리, 체인 구성, 에이전트 개발 등 다양한 기능을 제공합니다. 
또한 메모리 기능을 통해 대화 맥락을 유지하고, 다양한 도구와의 통합을 지원합니다.
"""

summary = summarization_chain.invoke({"text": text})
print("요약 결과:", summary)
print("요약 길이:", len(summary))

요약 결과: LangChain은 LLM 기반 애플리케이션 개발 프레임워크로, 데이터 연결, 프롬프트 관리, 체인 구성, 메모리 기능을 지원합니다.
요약 길이: 74


##### **2) 텍스트 요약 평가**

`(1) 요약 길이`

In [6]:
# 길이 평가도구 정의 
def evaluate_string_length(text, min_length=80, max_length=120):
    length = len(text)
    return {
        "score": min_length <= length <= max_length,
        "length": length
    }

# 길이 평가 수행
print(f"답변: {summary}")

# 길이 평가
result = evaluate_string_length(summary)
print(f"길이 평가 결과: {result}")

답변: LangChain은 LLM 기반 애플리케이션 개발 프레임워크로, 데이터 연결, 프롬프트 관리, 체인 구성, 메모리 기능을 지원합니다.
길이 평가 결과: {'score': False, 'length': 74}


`(2) ROUGE`

- **ROUGE(Recall-Oriented Understudy for Gisting Evaluation)**

    - 생성된 요약문의 품질을 평가

- ROUGE-1:
    - 참조 문서와 생성된 요약문 사이의 단일 단어(unigram) 중복을 측정 
    - 예시: "고양이가 잠을 잡니다"와 "고양이는 낮잠을 잡니다" 비교
        - 참조: ["고양이", "가", "잠", "을", "잡니다"]
        - 생성: ["고양이", "는", "낮잠", "을", "잡니다"]
        - 일치: ["고양이", "을", "잡니다"]
        - Precision = 3/5 = 0.6
        - Recall = 3/5 = 0.6
        - F1 = 0.6

- ROUGE-2:
    - 연속된 두 단어(bigram) 시퀀스의 중복을 평가
    - 예시: "고양이가 잠을 잡니다"와 "고양이는 낮잠을 잡니다" 비교
        - 참조: ["고양이 가", "가 잠", "잠 을", "을 잡니다"]
        - 생성: ["고양이 는", "는 낮잠", "낮잠 을", "을 잡니다"]
        - 일치: ["을 잡니다"]
        - Precision = 1/4 = 0.25
        - Recall = 1/4 = 0.25
        - F1 = 0.25

- ROUGE-L:
    - 최장 공통 부분수열(LCS)을 기반으로 텍스트의 유사도를 측정
    - 예시: "고양이가 잠을 잡니다"와 "고양이는 낮잠을 잡니다" 비교
        - 참조: "고양이 가 잠 을 잡니다"
        - 생성: "고양이 는 낮잠 을 잡니다"
        - LCS: "고양이 을 잡니다"
        - Precision = 3/5 = 0.6
        - Recall = 3/5 = 0.6
        - F1 = 0.6

- 각 메트릭은 정밀도(Precision), 재현율(Recall), F1-score로 표현
- ROUGE-1은 개별 단어 일치를, ROUGE-2는 문장 구조를, ROUGE-L은 전체적인 의미 유사성을 중점적으로 평가

In [7]:
from rouge_score import rouge_scorer
from kiwipiepy import Kiwi

# Kiwi 토크나이저 사용하여 토큰화하는 클래스 정의 
class CustomKiwiTokenizer(Kiwi):
    def tokenize(self, text):
        return [t.form for t in super().tokenize(text)]

# ROUGE 점수 계산 함수 정의
def calculate_rouge_score(reference, candidate):
    scorer = rouge_scorer.RougeScorer(
        ['rouge1', 'rouge2', 'rougeL'], 
        tokenizer=CustomKiwiTokenizer()    # 한국어 형태소 분석기 사용
    )
    scores = scorer.score(reference, candidate)
    return {
        "rouge1": scores['rouge1'].fmeasure,
        "rouge2": scores['rouge2'].fmeasure,
        "rougeL": scores['rougeL'].fmeasure
    }

# ROUGE 점수 계산
rouge_scores = calculate_rouge_score(text, summary)

print(f"ROUGE 점수: {rouge_scores}")

ROUGE 점수: {'rouge1': 0.36923076923076925, 'rouge2': 0.21875000000000003, 'rougeL': 0.3538461538461538}


`(3) LLM-as-Judge`

In [8]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

# LLM 초기화
llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)

# 평가를 위한 Pydantic 모델
class SummaryEvaluationResult(BaseModel):
    """요약 결과 평가"""
    summary_score: int = Field(description="요약의 품질 점수 (1-10)")
    summary_feedback: str = Field(description="요약에 대한 피드백")

# 평가 프롬프트 템플릿 정의
summary_evaluation_prompt = ChatPromptTemplate.from_messages([
    ("system", """다음의 원본 텍스트, 요약 결과를 평가하세요. 
    요약이 원본 텍스트의 핵심 내용을 얼마나 잘 담고 있는지 평가하고 
    1-10점 척도로 점수를 매기세요. 평가에 대한 피드백도 제공하세요."""),
    ("human", """
    원본 텍스트: {original_text}
    
    요약: {summary}
    """)
])

# 구조화된 출력을 위한 LLM 설정
summary_evaluation_llm = llm.with_structured_output(SummaryEvaluationResult)

# 평가 체인 생성
summary_evaluation_chain = summary_evaluation_prompt | summary_evaluation_llm

# 평가 체인 실행
evaluation_result = summary_evaluation_chain.invoke({
    "original_text": text,
    "summary": summary
})
print("평가 결과:", evaluation_result)
print("점수:", evaluation_result.summary_score)
print("피드백:", evaluation_result.summary_feedback)

평가 결과: summary_score=8 summary_feedback='요약은 LangChain의 주요 기능과 목적을 잘 담고 있습니다. 특히 LLM 기반 애플리케이션 개발, 데이터 연결, 프롬프트 관리, 체인 구성, 메모리 기능 지원을 명확히 언급하여 핵심 내용을 잘 전달하고 있습니다. 다만, 에이전트 개발과 다양한 도구와의 통합 지원 부분이 빠져 있어 완전한 요약으로 보기는 어렵습니다. 이 부분을 포함하면 더 완성도 높은 요약이 될 것입니다.'
점수: 8
피드백: 요약은 LangChain의 주요 기능과 목적을 잘 담고 있습니다. 특히 LLM 기반 애플리케이션 개발, 데이터 연결, 프롬프트 관리, 체인 구성, 메모리 기능 지원을 명확히 언급하여 핵심 내용을 잘 전달하고 있습니다. 다만, 에이전트 개발과 다양한 도구와의 통합 지원 부분이 빠져 있어 완전한 요약으로 보기는 어렵습니다. 이 부분을 포함하면 더 완성도 높은 요약이 될 것입니다.


---

### **[실습]**

- 프롬프트를 수정하여 요약 체인을 정의합니다. 
- 뉴스 데이터를 사용하여 요약을 생성합니다. 
- 요약 결과를 평가합니다. 

In [ ]:
# 여기에 코드를 작성하세요. 

---

### 2. **키워드 추출 (Extraction)**

- **키워드 추출**은 문서에서 가장 중요한 용어와 개념을 식별하는 과정임
- 키워드 추출을 통해 문서의 **핵심 주제와 내용**을 빠르게 파악할 수 있음
- 추출된 키워드는 문서 **분류, 검색, 인덱싱**에 활용됨

##### **1) 키워드 추출 체인**

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from typing import List

# LLM 초기화
llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)

# 키워드 추출을 위한 Pydantic 모델
class Keywords(BaseModel):
    """텍스트에서 추출된 키워드 목록"""
    keywords: List[str] = Field(description="텍스트에서 추출된 중요 키워드 목록 ")

# 키워드 추출 프롬프트 템플릿 정의
keyword_extraction_prompt = ChatPromptTemplate.from_messages([
    ("system", "다음 텍스트에서 중요한 키워드를 추출하세요. 키워드는 텍스트의 핵심 주제와 개념을 나타내야 합니다. (최대 3개)"),
    ("human", "{text}")
])

# 구조화된 출력을 위한 LLM 설정
keyword_extraction_llm = llm.with_structured_output(Keywords)

# 키워드 추출 체인 생성
keyword_extraction_chain = keyword_extraction_prompt | keyword_extraction_llm

# 예시 사용
keywords_result = keyword_extraction_chain.invoke({"text": text})
print("키워드 추출 결과:", keywords_result.keywords)

키워드 추출 결과: ['LangChain', '대규모 언어 모델(LLM)', '프레임워크']


##### **2) 키워드 추출 평가**

`(1) 키워드 개수`

In [10]:
# 평가도구 정의 
def evaluate_keyword_count(keywords, min_count=1, max_count=3):
    """키워드 개수를 평가하는 함수"""
    count = len(keywords)
    return {
        "score": min_count <= count <= max_count,
        "count": count
    }

# 평가 수행
print(f"답변: {keywords_result.keywords}")

# 키워드 개수 평가
result = evaluate_keyword_count(keywords_result.keywords)
print(f"키워드 개수 평가 결과: {result}")

답변: ['LangChain', '대규모 언어 모델(LLM)', '프레임워크']
키워드 개수 평가 결과: {'score': True, 'count': 3}


`(2) LLM-as-Judge`

In [11]:
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model

# LLM 초기화
llm = init_chat_model("openai:gpt-4.1-mini", temperature=0)

# 평가를 위한 Pydantic 모델
class KeywordEvaluationResult(BaseModel):
    """키워드 추출 결과 평가"""
    keyword_score: int = Field(description="키워드의 품질 점수 (1-10)")
    keyword_feedback: str = Field(description="키워드에 대한 피드백")

# 평가 프롬프트 템플릿 정의
keyword_evaluation_prompt = ChatPromptTemplate.from_messages([
    ("system", """다음의 원본 텍스트, 키워드 추출 결과를 평가하세요. 
    키워드가 원본 텍스트의 핵심 내용을 얼마나 잘 담고 있는지 평가하고 
    1-10점 척도로 점수를 매기세요. 평가에 대한 피드백도 제공하세요."""),
    ("human", """
    원본 텍스트: {original_text}

    키워드: {keywords}
    """)
])

# 구조화된 출력을 위한 LLM 설정
keyword_evaluation_llm = llm.with_structured_output(KeywordEvaluationResult)

# 평가 체인 생성
keyword_evaluation_chain = keyword_evaluation_prompt | keyword_evaluation_llm

# 평가 체인 실행
evaluation_result = keyword_evaluation_chain.invoke({
    "original_text": text,
    "keywords": keywords_result.keywords
})

print("평가 결과:", evaluation_result)
print("점수:", evaluation_result.keyword_score)
print("피드백:", evaluation_result.keyword_feedback)

평가 결과: keyword_score=7 keyword_feedback='키워드가 LangChain과 대규모 언어 모델(LLM), 프레임워크라는 핵심 개념을 잘 포함하고 있어 기본적인 내용을 잘 반영하고 있습니다. 다만, 프롬프트 관리, 체인 구성, 에이전트 개발, 메모리 기능, 외부 데이터 소스 연결 등 LangChain의 구체적인 기능과 특징들이 키워드에 포함되지 않아 내용의 깊이나 다양성을 충분히 담아내지 못했습니다. 좀 더 상세한 기능이나 특징을 키워드에 추가하면 원본 텍스트의 핵심 내용을 더 잘 반영할 수 있을 것입니다.'
점수: 7
피드백: 키워드가 LangChain과 대규모 언어 모델(LLM), 프레임워크라는 핵심 개념을 잘 포함하고 있어 기본적인 내용을 잘 반영하고 있습니다. 다만, 프롬프트 관리, 체인 구성, 에이전트 개발, 메모리 기능, 외부 데이터 소스 연결 등 LangChain의 구체적인 기능과 특징들이 키워드에 포함되지 않아 내용의 깊이나 다양성을 충분히 담아내지 못했습니다. 좀 더 상세한 기능이나 특징을 키워드에 추가하면 원본 텍스트의 핵심 내용을 더 잘 반영할 수 있을 것입니다.


---

### **[실습]**

- 프롬프트를 수정하여 키워드 추출 체인을 정의합니다. 
- 뉴스 데이터를 사용하여 키워드 추출 작업을 수행합니다. 
- 추출 결과를 평가합니다. 

In [ ]:
# 여기에 코드를 작성하세요. 